In [1]:
from glob import glob
from random import sample
import json
from Levenshtein import distance as levenshtein_distance
import re

import texte


path = "Corpus/Mazarinades/*/*.xml"

files = glob(path)

print(len(files))

sample_mode = True

if sample_mode:
    files = sample(files, 200)
#     collection_textes = [texte.Texte(file) for file in tqdm(files)]
# else:
collection_textes = [e for e in texte.corpora(files) if e.plain]

In [3]:
with open("lexiques/LGERM.json", "r") as f:
    lexique = json.load(f)

lexique = set(e.lower() for e in lexique)

chriffre_romain = re.compile(r"[IVXLCDM]+\.?")

def corrige_mot(mot):
    if chriffre_romain.fullmatch(mot):
        return (mot, "chiffre romain")

    if mot == "&amp;":
        return ("&", "ampersand")

    if mot.lower() in lexique:
        return (mot, "lexique")

    return (min(lexique, key=lambda x: levenshtein_distance(x, mot.lower())), "levenshtein")

with open("extremes/lexicalite_max.txt", "r") as f:
    hapaxmax = f.read()

hapaxmax = "\n".join(hapaxmax.split("\n")[3:])
print(hapaxmax)

for mot in hapaxmax.split():
    correction, type_ = corrige_mot(mot)
    if type_ != "lexique":
        print(type_, mot, correction)


LE GENEREVX PRINCE AVX PARISIENS DE MOURIR POUR LE SERVICE DV ROY & de ne point ſouffrir le retour du Cardinal Mazarin. A PARIS Chez NICOLAS L'ANGEVIN. M. DC. LII. 
chiffre romain DV DV
levenshtein ſouffrir souffrir
levenshtein Mazarin. mazarin
levenshtein L'ANGEVIN. angevine
chiffre romain M. M.
chiffre romain DC. DC.
chiffre romain LII. LII.
